In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
%pip install pytorch-lightning -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.6 MB/s eta 0:00:00


In [2]:

def parse_conll_file(file_path):
    """
    Parse a conll-formatted file and return a list of sentences, where each sentence is a list of tokens and their labels.

    Args:
        file_path (str): The path to the conll file.

    Returns:
        List[List[Tuple[str, str]]]: A list of sentences, where each sentence is a list of (token, label) tuples.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.read().strip().split('\n\n')

        sentences = []
        for line in lines:
            sentence = []
            # print(line)
            flag = 0
            for token in line.strip().split('\n'):
                flag = flag + 1
                if(flag==1):
                  continue
                fields = token.split(' _ _ ')
                sentence.append((fields[0], fields[-1]))
            if sentence:
                sentences.append(sentence)
    return sentences

In [3]:
train_dataset = parse_conll_file('en_train.conll')
dev_dataset = parse_conll_file('en_dev.conll')
test_dataset = parse_conll_file('en_test.conll')

In [4]:
print(train_dataset[0])

[('robert', 'B-OtherPER'), ('gottschalk', 'I-OtherPER'), ('1939', 'O'), ('academy', 'B-VisualWork'), ('award', 'I-VisualWork'), ('winner', 'O'), ('and', 'O'), ('founder', 'O'), ('of', 'O'), ('panavision', 'B-ORG')]


In [5]:
# print(train_dataset)
# print(test_dataset)
dev_dataset = parse_conll_file('en_dev.conll')
print(dev_dataset)

[[('eli', 'B-OtherPER'), ('lilly', 'I-OtherPER'), ('founder', 'O'), ('president', 'O'), ('of', 'O'), ('pharmaceutical', 'O'), ('company', 'O'), ('eli', 'B-PublicCorp'), ('lilly', 'I-PublicCorp'), ('and', 'I-PublicCorp'), ('company', 'I-PublicCorp')], [('christoph', 'B-OtherPER'), ('haberland', 'I-OtherPER'), ('designed', 'O'), ('a', 'O'), ('new', 'O'), ('marble', 'O'), ('pulpit', 'B-OtherPROD'), ('for', 'O'), ('the', 'O'), ('church', 'O'), ('which', 'O'), ('was', 'O'), ('built', 'O'), ('in', 'O'), ('italy', 'B-HumanSettlement'), ('in', 'O'), ('1793', 'O'), ('.', 'O')], [('he', 'O'), ('was', 'O'), ('succeeded', 'O'), ('as', 'O'), ('chancellor', 'O'), ('by', 'O'), ('sir', 'B-OtherPER'), ('frank', 'I-OtherPER'), ('kitto', 'I-OtherPER'), ('.', 'O')], [('it', 'O'), ('was', 'O'), ('described', 'O'), ('by', 'O'), ('edward', 'B-OtherPER'), ('meyrick', 'I-OtherPER'), ('in', 'O'), ('1915', 'O'), ('.', 'O')], [('having', 'O'), ('suffered', 'O'), ('depredation', 'O'), ('of', 'O'), ('the', 'O'), ('

In [6]:
# print(torch.utils.data.Dataset(dev_dataset))
# dev_dataset = DataLoader(dev_dataset)

In [7]:
print(dev_dataset)
# print(list(torch.utils.data.DataLoader(dev_dataset)))

[[('eli', 'B-OtherPER'), ('lilly', 'I-OtherPER'), ('founder', 'O'), ('president', 'O'), ('of', 'O'), ('pharmaceutical', 'O'), ('company', 'O'), ('eli', 'B-PublicCorp'), ('lilly', 'I-PublicCorp'), ('and', 'I-PublicCorp'), ('company', 'I-PublicCorp')], [('christoph', 'B-OtherPER'), ('haberland', 'I-OtherPER'), ('designed', 'O'), ('a', 'O'), ('new', 'O'), ('marble', 'O'), ('pulpit', 'B-OtherPROD'), ('for', 'O'), ('the', 'O'), ('church', 'O'), ('which', 'O'), ('was', 'O'), ('built', 'O'), ('in', 'O'), ('italy', 'B-HumanSettlement'), ('in', 'O'), ('1793', 'O'), ('.', 'O')], [('he', 'O'), ('was', 'O'), ('succeeded', 'O'), ('as', 'O'), ('chancellor', 'O'), ('by', 'O'), ('sir', 'B-OtherPER'), ('frank', 'I-OtherPER'), ('kitto', 'I-OtherPER'), ('.', 'O')], [('it', 'O'), ('was', 'O'), ('described', 'O'), ('by', 'O'), ('edward', 'B-OtherPER'), ('meyrick', 'I-OtherPER'), ('in', 'O'), ('1915', 'O'), ('.', 'O')], [('having', 'O'), ('suffered', 'O'), ('depredation', 'O'), ('of', 'O'), ('the', 'O'), ('

In [8]:
SEQ_LEN = 25

# Create word_to_idx and tag_to_idx mappings
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
tag_to_idx = {"<PAD>": 0}


def preprocess(dataset):
    # Extract sentences and tags
    sent = [[token.lower() for token, tag in sentence] for sentence in dataset]
    tags = [[tag for token, tag in sentence] for sentence in dataset]

    for i in range(len(sent)):
        while len(sent[i]) < SEQ_LEN:
            sent[i].append('<PAD>')
            tags[i].append('<PAD>')

        if len(sent[i]) > SEQ_LEN:
            sent[i] = sent[i][:SEQ_LEN]
            tags[i] = tags[i][:SEQ_LEN]
    
    for sentence_tags in tags:
        for tag in sentence_tags:
            if tag not in tag_to_idx:
                tag_to_idx[tag] = len(tag_to_idx)
    
    for sentence in sent:
        for word in sentence:
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    # Convert words and tags to indices
    X = torch.tensor([[word_to_idx.get(word, 1) for word in sentence] for sentence in sent], dtype=torch.int).type(torch.LongTensor)
    Y = torch.tensor([[tag_to_idx[tag] for tag in sentence] for sentence in tags], dtype=torch.int).type(torch.LongTensor)
    
    return X, Y


In [9]:
train_X, train_Y = preprocess(train_dataset)
dev_X, dev_Y = preprocess(dev_dataset)
test_X, test_Y = preprocess(test_dataset)

In [10]:
print(tag_to_idx["O"])

3


In [11]:
print(f"Number of training examples: {len(train_X)}")
print(f"Number of validation examples: {len(dev_X)}")
print(f"Number of testing examples: {len(test_X)}")

Number of training examples: 16778
Number of validation examples: 871
Number of testing examples: 249980


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class POSModel(pl.LightningModule):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, num_layers=1, bidirectional=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) #B * seq_len, B * seq_len * embedding_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=bidirectional)
        #B * seq_len * embedding_dim -> B * seq_len * hidden_dim 
        #tags
        if bidirectional:
            self.fc = nn.Linear(2*hidden_dim, tagset_size)
        else:
            self.fc = nn.Linear(hidden_dim, tagset_size)
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        embeds = self.embedding(x)
        #print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.fc(lstm_out)
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss_fn(y_hat.view(-1, y_hat.shape[-1]), y.view(-1))
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

In [13]:
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

EMBEDDING_DIM = 200
HIDDEN_DIM    = 200
NUM_EPOCHS    = 10 
BATCH_SIZE    = 15

train_dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(dev_X, dev_Y)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [14]:
model = POSModel(vocab_size=len(word_to_idx), tagset_size=len(tag_to_idx), embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, bidirectional=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")
# trainer = pl.Trainer(max_epochs=NUM_EPOCHS, gpus=1, callbacks=[early_stopping])
trainer = pl.Trainer(max_epochs=NUM_EPOCHS,  callbacks=[early_stopping])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.9/dist-packages/lightning_fabric/accelerators/cuda.py:235: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 48.3 M
1 | lstm      | LSTM             | 643 K 
2 | fc        | Linear           | 27.3 K
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
49.0 M    Trainable params

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(


ValueError: ignored

In [ ]:
from sklearn.metrics import classification_report

# define idx_to_tag
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

# define device
device = torch.device('cpu')

# Create a dataloader for the test set
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Set the model to evaluation mode
model.eval()

y_true = []
y_pred = []
i=0

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)
        if i<10:
          print(y_hat.shape)
          i+=1
        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]
        
        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]

print(classification_report(y_true, y_pred))

In [ ]:
# Set the model to evaluation mode
model.eval()

idx_to_word = {idx: word for word, idx in word_to_idx.items()}

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # Move the data to the device
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        y_hat = model(x)

        # Get back the sentence
        x_sent = [idx_to_word[i] for i in x.cpu().numpy().flatten().tolist()]

        # Compute the predicted tags
        y_pred += [idx_to_tag[i] for i in y_hat.argmax(-1).cpu().numpy().flatten().tolist()]

        # Compute the true tags
        y_true += [idx_to_tag[i] for i in y.cpu().numpy().flatten().tolist()]
        print("Sentence")
        print(x_sent)
        print("Predicted tags")
        print(y_pred)
        break